In [ ]:
from qctoolkit.hardware.setup import HardwareSetup, PlaybackChannel, MarkerChannel
hardware_setup = HardwareSetup()

In [4]:
from qctoolkit.hardware.awgs.tabor import TaborChannelPair, TaborAWGRepresentation

tabor_address = r'USB0::0x168C::0x2184::0000216488::INSTR'

# small wrapper around pytabor
tawg = TaborAWGRepresentation(r'USB0::0x168C::0x2184::0000216488::INSTR', reset=True)
tawg.paranoia_level = 2

# actual awg driver object
tchannelpair = TaborChannelPair(tawg, (1, 2), 'TABOR_AB')

Failed to open "USB0::0x168C::0x2184::0000216488::INSTR"
(<class 'pyvisa.errors.VisaIOError'>, VisaIOError('VI_ERROR_RSRC_NFOUND (-1073807343): Insufficient location information or the requested device or resource is not present in the system.',), <traceback object at 0x0000018F5D498CC8>)


AttributeError: 'NoneType' object has no attribute 'write'

In [ ]:
hardware_setup.set_channel('TABOR_A', PlaybackChannel(tchannelpair, 0))
hardware_setup.set_channel('TABOR_B', PlaybackChannel(tchannelpair, 1))
hardware_setup.set_channel('TABOR_A_MARKER', MarkerChannel(tchannelpair, 0))
hardware_setup.set_channel('TABOR_B_MARKER', MarkerChannel(tchannelpair, 1))

In [ ]:
import atsaverage
import atsaverage.atsaverage
atsaverage.atsaverage.Logger.log_path = r'C:\Users\Public\AtsAverageLog'
atsaverage.atsaverage.Logger.rotate_file()

ModuleNotFoundError: No module named 'atsaverage'

In [ ]:
import atsaverage.atsaverage
atsaverage.atsaverage.Logger.log_path = r'C:\Users\Public\AtsAverageLog'
atsaverage.atsaverage.Logger.rotate_file()
from qctoolkit.hardware.dacs.alazar import AlazarCard

alazar = AlazarCard(atsaverage.core.getLocalCard(1, 1))

# mask MyMask will be applied/refers to data from channel 0
alazar.register_mask_for_channel('MyMask', 0)



In [3]:
def get_alazar_config():
    from atsaverage import alazar
    from atsaverage.config import ScanlineConfiguration, CaptureClockConfiguration, EngineTriggerConfiguration,\
        TRIGInputConfiguration, InputConfiguration
        
    r = 2.5
    rid = alazar.TriggerRangeID.etr_2V5

    trig_level = int((r + 0.15) / (2*r) * 255)
    assert 0 <= trig_level < 256

    config = ScanlineConfiguration()
    config.triggerInputConfiguration = TRIGInputConfiguration(triggerRange=rid)
    config.triggerConfiguration = EngineTriggerConfiguration(triggerOperation=alazar.TriggerOperation.J,
                                                             triggerEngine1=alazar.TriggerEngine.J,
                                                             triggerSource1=alazar.TriggerSource.external,
                                                             triggerSlope1=alazar.TriggerSlope.positive,
                                                             triggerLevel1=trig_level,
                                                             triggerEngine2=alazar.TriggerEngine.K,
                                                             triggerSource2=alazar.TriggerSource.disable,
                                                             triggerSlope2=alazar.TriggerSlope.positive,
                                                             triggerLevel2=trig_level)
    config.captureClockConfiguration = CaptureClockConfiguration(source=alazar.CaptureClockType.internal_clock,
                                                                 samplerate=alazar.SampleRateID.rate_100MSPS)
    config.inputConfiguration = 4*[InputConfiguration(input_range=alazar.InputRangeID.range_1_V)]
    config.totalRecordSize = 0
    config.aimedBufferSize = 10*config.aimedBufferSize

    # is set automatically
    assert config.totalRecordSize == 0
    
    return config

In [ ]:
alazar.config = get_alazar_config()